# Bokeh: Interactions

Most parts of this tutorial are taken from the bokeh [website](https://docs.bokeh.org/en/latest/index.html).

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

From previous chapters we know how multiple plots can be placed together in a layout. Now we can start to look at how different plots can be linked together, or how plots can be linked to widgets. 

# Linked Interactions

It is possible to link various interactions between different Bokeh plots. For instance, the ranges of two (or more) plots can be linked, so that when one of the plots is panned (or zoomed, or otherwise has its range changed) the other plots will update in unison. It is also possible to link selections between two plots, so that when items are selected on one plot, the corresponding items on the second plot also become selected. 

## Linked panning

Linked panning (when multiple plots have ranges that stay in sync) is simple to achieve with Bokeh. You simply share the appropriate range objects between two (or more) plots. The example below shows how to accomplish this by linking the ranges of three plots in various ways:

In [ ]:
from bokeh.layouts import gridplot

x = list(range(11))
y0, y1, y2 = x, [10-i for i in x], [abs(i-5) for i in x]

plot_options = dict(width=250, plot_height=250, tools='pan,wheel_zoom')

# create a new plot
s1 = figure(**plot_options)
s1.circle(x, y0, size=10, color="navy")

# create a new plot and share both ranges
s2 = figure(x_range=s1.x_range, y_range=s1.y_range, **plot_options)
s2.triangle(x, y1, size=10, color="firebrick")

# create a new plot and share only one range
s3 = figure(x_range=s1.x_range, **plot_options)
s3.square(x, y2, size=10, color="olive")

p = gridplot([[s1, s2, s3]])

# show the results
show(p)

## Linked brushing

Linking selections is accomplished in a similar way, by sharing data sources between plots. Note that normally with ``bokeh.plotting`` and ``bokeh.charts`` creating a default data source for simple plots is handled automatically. However to share a data source, we must create them by hand and pass them explicitly. This is illustrated in the example below:

In [ ]:
from bokeh.models import ColumnDataSource

x = list(range(-20, 21))
y0, y1 = [abs(xx) for xx in x], [xx**2 for xx in x]

# create a column data source for the plots to share
source = ColumnDataSource(data=dict(x=x, y0=y0, y1=y1))

TOOLS = "box_select,lasso_select,help"

# create a new plot and add a renderer
left = figure(tools=TOOLS, width=300, height=300)
left.circle('x', 'y0', source=source)

# create another new plot and add a renderer
right = figure(tools=TOOLS, width=300, height=300)
right.circle('x', 'y1', source=source)

p = gridplot([[left, right]])

show(p)

<h3>
<font color='blue'>
Exercise
</font>
</h3>
Create two plots in a gridplot, and link their data sources

# Hover Tools

Bokeh has a Hover Tool that allows additional information to be displayed in a popup whenever the user hovers over a specific glyph. Basic hover tool configuration amounts to providing a list of ``(name, format)`` tuples. The full details can be found in the User's Guide [here](https://bokeh.pydata.org/en/latest/docs/user_guide/tools.html#hovertool).

The example below shows some basic usage of the Hover tool with a circle glyph, using hover information defined in utils.py:

In [ ]:
from bokeh.models import HoverTool

source = ColumnDataSource(
        data=dict(
            x=[1, 2, 3, 4, 5],
            y=[2, 5, 8, 2, 7],
            desc=['A', 'b', 'C', 'd', 'E'],
        )
    )

hover = HoverTool(
        tooltips=[
            ("index", "$index"),
            ("(x,y)", "($x, $y)"),
            ("desc", "@desc"),
        ]
    )

p = figure(plot_width=300, plot_height=300, tools=[hover], title="Mouse over the dots")

p.circle('x', 'y', size=20, source=source)

show(p)

# Widgets

Bokeh supports direct integration with a small basic widget set. Thse can be used in conjunction with a Bokeh Server (or with ``CustomJS`` models, which we won't use in this tutorial) to add more interactive capability to your documents. You can see a complete list, with example code in the [Adding Widgets](https://bokeh.pydata.org/en/latest/docs/user_guide/interaction.html#adding-widgets) section of the User's Guide.

To use the widgets, include them in a layout like you would a plot object:

In [ ]:
from bokeh.models.widgets import Slider
from bokeh.models import Column


slider = Slider(start=0, end=10, value=1, step=.1, title="foo")

show(Column(slider))

<h3>
<font color='blue'>
Exercise
</font>
</h3>
Create and show a Select widget 


# Bokeh server/Running Bokeh Applications 

In order for a widget to be useful, it needs to be able to perform some action. Using the Bokeh server, it is possible to have widgets trigger real Python code.

The architecture of Bokeh is such that high-level “model objects” (representing things like plots, ranges, axes, glyphs, etc.) are created in Python, and then converted to a JSON format that is consumed by the client library, BokehJS. Using the Bokeh Server, it is possible to keep the “model objects” in python and in the browser in sync with one another, creating powerful capabilities:

* respond to UI and tool events generated in a browser with computations or queries using the full power of python
* automatically push updates the UI (i.e. widgets or plots), in a browser
* use periodic, timeout, and asychronous callbacks drive streaming updates

From Wikipedia: In computer programming, a callback, also known as a "call-after" function, is any executable code that is passed as an argument to other code; that other code is expected to call back (execute) the argument at a given time. 

***This capability to synchronize between python and the browser is the main purpose of the Bokeh Server.***

In [ ]:
from bokeh.io import output_notebook, show
output_notebook()

## Bokeh Apps in Notebooks

The easiest way to embed a Bokeh application in a notebook is to make a function `modify_doc(doc)` that creates Bokeh content, and adds it to the document. This function can be passed to `show`, and the app defined by the function will be displayed inline. A short complete example is below 

In [ ]:
from bokeh.layouts import column
from bokeh.models import TextInput, Button, Paragraph

def modify_doc(doc):
    
    # create some widgets
    button = Button(label="Say HI")
    input = TextInput(value="Bokeh")
    output = Paragraph()

    # add a callback to a widget
    def update():
        output.text = "Hello, " + input.value
    button.on_click(update)

    # create a layout for everything
    layout = column(button, input, output)

    # add the layout to curdoc
    doc.add_root(layout)
    
# In the notebook, just pass the function that defines the app to show
# You may need to supply notebook_url, e.g notebook_url="http://localhost:8889" 
show(modify_doc) 

## Bokeh Apps with `bokeh serve` 

It's also possible to define Bokeh applications by creating a standard Python script. In this case, there is no need to make a function like `modify_doc`. Typically, the script should simply create all the bokeh content, then add it to the doc with a line like
```python
curdoc().add_root(layout)
```

To try out the example below, copy the code into a file ``hello.py`` and then execute:
```bash
bokeh serve --show hello.py 
```

<center><div style="font-size: 14pt;color: firebrick;"> NOTE: The exercise below requires work outside the notebook <div></center>

```python
# hello.py 

from bokeh.io import curdoc
from bokeh.layouts import column
from bokeh.models.widgets import TextInput, Button, Paragraph

# create some widgets
button = Button(label="Say HI")
input = TextInput(value="Bokeh")
output = Paragraph()

# add a callback to a widget
def update():
    output.text = "Hello, " + input.value
button.on_click(update)

# create a layout for everything
layout = column(button, input, output)

# add the layout to curdoc
curdoc().add_root(layout)
```

Copy this code to a script `hello.py` and run it with the Bokeh server.

## Linking Plots and Widgets

Lets take a look at a more involved example that links several widgets to a plot. 

Before we continue it is important to know that all interactive elements in a webpage (and also from a `bokeh serve` page) are made interactive with JavaScript. JavaScript is another programming language which works quite different from what you're used to with Python. The JavaScript programming language is used specifically for webpages. 

This *bokeh* library, and most other visualization tools that are similar to *bokeh*, are based on JavaScript and therefore sometimes use the logic from JavaScript (but of course, Python syntax). If you know JavaScript, it is probably a bit easier to understand what every element does. 

The most important JavaScript principle that is used here is a callback. A callback is when you give a function definition with some arguments in your code, which you expect to be called later on again. In this example, the callback function is `update_color` and the moment when this function is called is *when the widget changes*, which happens when the user selects another color. This function is then called with what `attr`ibute changes (in this case always `"value"`), the previous value `old` from before the change, and the new value `new`. This callback is added to the widget with the function `on_change`, which in turn calls the function every time something changes in the widget, with exactly those three arguments. Below this example, you can find a simpler callback example.

In [ ]:
from numpy.random import random

from bokeh.layouts import column, row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Select, TextInput

def get_data(N):
    """ Creates N random datapoints with a random x, y, and radius. """
    return dict(x=random(size=N), y=random(size=N), r=random(size=N) * 0.03)

COLORS = ["black", "firebrick", "navy", "olive", "goldenrod"]

def modify_doc(doc):
    # first get starting data for this visualization using the function defined above
    source = ColumnDataSource(data=get_data(200))

    # create our graph with the data loaded above
    p = figure(tools="", toolbar_location=None)
    r = p.circle(x='x', y='y', radius='r', source=source,
                 color="navy", alpha=0.6, line_color="white")

    # create an input selector and textbox for the user, set the default values
    select = Select(title="Color", value="navy", options=COLORS)
    input = TextInput(title="Number of points", value="200")

    # the callback function that will change the color of the glyphs depending on the color selector
    def update_color(attrname, old, new):
        r.glyph.fill_color = select.value
    
    # when something in the selector changes, call the callback function that changes the colors of the datapoints
    select.on_change('value', update_color)

    # the callback function that will change the number of points by getting new random points
    def update_points(attrname, old, new):
        N = int(input.value)
        source.data = get_data(N)
    
    # couple the callback function to the text field
    input.on_change('value', update_points)

    # define how we want to position our selectors and figure
    layout = column(row(select, input, width=400), row(p))

    # place the layout on the document page
    doc.add_root(layout)

show(modify_doc)

### A simpler callback example

Below you can find a short example of another callback function. In this case, when the button is clicked, we print something. (Disclaimer: usually you do not want to print stuff if you change widgets, most of the times you use this to update your plot!) 

In [ ]:
def modify_doc(doc):
    
    # create button
    button = Button(label="This is a button")
    
    # in this function we decide what we would like to do when the button is clicked
    def update():
        print('You clicked the button!')
        
    # specify that we would like to run update() when the button is clicked
    button.on_click(update)

    layout = column(button)
    doc.add_root(layout)

show(modify_doc)

<h3>
<font color='blue'>
Exercise
</font>
</h3>
Add more widgets to change more aspects of the plot in the 'Linking plots and widgets' section' (above)
